In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_matching/snli/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [3]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
# stream data from text files
def data_generator(f_path, word2idx):
  label2idx = {'neutral': 0, 'entailment': 1, 'contradiction': 2,}
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      label, text1, text2 = line.split('\t')
      if label == '-':
        continue
      text1 = [word2idx.get(w, len(word2idx)) for w in text1.split()]
      text2 = [word2idx.get(w, len(word2idx)) for w in text2.split()]
      yield ((text1, text2), label2idx[label])


def dataset(is_training, params):
  _shapes = (([None], [None]), ())
  _types = ((tf.int32, tf.int32), tf.int32)
  _pads = ((0, 0), -1)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params['word2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params['word2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  return ds

In [0]:
def masked_attention(x, align, mask, tile_len):
  pad = tf.fill(tf.shape(align), float('-inf'))
  mask = tf.tile(tf.expand_dims(mask, 1), [1, tile_len, 1])
  align = tf.where(tf.equal(mask, 0), pad, align)
  align = tf.nn.softmax(align)
  return tf.matmul(align, x)


def soft_align_attention(x1, x2, mask1, mask2):
  align12 = tf.matmul(x1, x2, transpose_b=True)
  align21 = tf.transpose(align12, [0,2,1])
  x1_ = masked_attention(x2, align12, mask2, tf.shape(x1)[1])
  x2_ = masked_attention(x1, align21, mask1, tf.shape(x2)[1])
  return x1_, x2_

In [0]:
class AttentivePooling(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    self.dropout = tf.keras.layers.Dropout(.2)
    self.kernel = tf.keras.layers.Dense(units=1,
                                        activation=tf.tanh,
                                        use_bias=False)

  
  def call(self, inputs, training=False):
    x, masks = inputs
    # alignment
    align = tf.squeeze(self.kernel(self.dropout(x, training=training)), -1)
    # masking
    paddings = tf.fill(tf.shape(align), float('-inf'))
    align = tf.where(tf.equal(masks, 0), paddings, align)
    # probability
    align = tf.nn.softmax(align)
    align = tf.expand_dims(align, -1)
    # weighted sum
    return tf.squeeze(tf.matmul(x, align, transpose_a=True), -1)

In [0]:
class ESIM(tf.keras.Model):
  def __init__(self, params: dict):
    super().__init__()
    self.embedding = tf.Variable(np.load(params['embedding_path']), name='pretrained_embedding')
    
    self.input_dropout = tf.keras.layers.Dropout(.2)
    self.input_encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        params['rnn_units'], return_sequences=True), name='input_encoder')
    
    self.feature_dropout = tf.keras.layers.Dropout(.5)
    self.feature_fc = tf.keras.layers.Dense(params['rnn_units'], tf.nn.elu, name='feature_fc')
    
    self.revise_dropout = tf.keras.layers.Dropout(.2)
    self.revise_encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        params['rnn_units'], return_sequences=True), name='revise_encoder')
    
    self.attentive_pooling = AttentivePooling(params)
    
    self.fc1_dropout = tf.keras.layers.Dropout(.5)
    self.fc1 = tf.keras.layers.Dense(params['rnn_units'], tf.nn.elu, name='final_fc1')
    self.fc2_dropout = tf.keras.layers.Dropout(.2)
    self.fc2 = tf.keras.layers.Dense(params['rnn_units'], tf.nn.elu, name='final_fc2')
    
    self.out_linear = tf.keras.layers.Dense(params['num_labels'], name='out_linear')
  
  
  def call(self, inputs, training=False):
    x1, x2 = inputs
    
    if x1.dtype != tf.int32:
      x1 = tf.cast(x1, tf.int32)
    if x2.dtype != tf.int32:
      x2 = tf.cast(x2, tf.int32)
    
    batch_sz = tf.shape(x1)[0]
    
    mask1 = tf.sign(x1)
    mask2 = tf.sign(x2)
    
    x1 = tf.nn.embedding_lookup(self.embedding, x1)
    x2 = tf.nn.embedding_lookup(self.embedding, x2)
    
    self.input_dropout.noise_shape = (batch_sz, 1, 300)
    x1 = self.input_dropout(x1, training=training)
    x2 = self.input_dropout(x2, training=training)
    
    x1 = self.input_encoder(x1, mask=None)
    x2 = self.input_encoder(x2, mask=None)
    
    x1_, x2_ = soft_align_attention(x1, x2, mask1, mask2)
    aggregate_fn = lambda x, x_: tf.concat((x,
                                            x_,
                                           (x - x_),
                                           (x * x_),), -1)
    x1 = aggregate_fn(x1, x1_)
    x2 = aggregate_fn(x2, x2_)
    
    self.feature_dropout.noise_shape = (batch_sz, 1, params['rnn_units']*8)
    x1 = self.feature_dropout(x1, training=training)
    x2 = self.feature_dropout(x2, training=training)
    
    x1 = self.feature_fc(x1)
    x2 = self.feature_fc(x2)
    
    self.revise_dropout.noise_shape = (batch_sz, 1, params['rnn_units'])
    x1 = self.revise_dropout(x1, training=training)
    x2 = self.revise_dropout(x2, training=training)
    
    x1 = self.revise_encoder(x1, mask=None)
    x2 = self.revise_encoder(x2, mask=None)
    
    features = []
    features.append(tf.reduce_max(x1, axis=1))
    features.append(tf.reduce_max(x2, axis=1))
    features.append(self.attentive_pooling((x1, mask1), training=training))
    features.append(self.attentive_pooling((x2, mask2), training=training))
    x = tf.concat(features, axis=-1)
    
    x = self.fc1_dropout(x, training=training)
    x = self.fc1(x)
    
    x = self.fc2_dropout(x, training=training)
    x = self.fc2(x)
    
    x = self.out_linear(x)
    
    return x

In [0]:
params = {
    'model_dir': '../model/esim',
    'log_path': '../log/esim.txt',
    'train_path': '../data/train.txt',
    'test_path': '../data/test.txt',
    'embedding_path': '../vocab/word.npy',
    'vocab_path': '../vocab/word.txt',
    'batch_size': 32,
    'buffer_size': 200000,
    'rnn_units': 300,
    'lr': 4e-4,
    'clip_norm': 10.,
    'num_labels': 3,
    'eval_steps': 550152//32//10,
    'num_patience': 7,
}

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [11]:
params['word2idx'] = get_vocab(params['vocab_path'])
params['vocab_size'] = len(params['word2idx']) + 1

model = ESIM(params)
model.build([[None, None], [None, None]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.99)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

while True:
  # TRAINING
  for ((text1, text2), label) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model((text1, text2), training=True)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logits)
      loss = tf.reduce_mean(loss)
      
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, model.trainable_variables))
    
    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
    if global_step % params['eval_steps'] == 0:
      # EVALUATION
      m = tf.keras.metrics.Accuracy()

      for ((text1, text2), label) in dataset(is_training=False, params=params):
        logits = model((text1, text2), training=False)
        y_pred = tf.argmax(logits, axis=-1)
        m.update_state(y_true=label, y_pred=y_pred)

      acc = m.result().numpy()
      logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))
      history_acc.append(acc)

      if acc > best_acc:
        best_acc = acc
        # you can save model here
      logger.info("Best Accuracy: {:.3f}".format(best_acc))

      if len(history_acc) > params['num_patience'] and is_descending(history_acc):
        logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
        break

W0411 01:07:38.346787 139703861815168 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f0ee043f710>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0411 01:07:38.360240 139703861815168 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f0ee043fc88>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0411 01:07:38.371503 139703861815168 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f0ee03fb940>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0411 01:07:38.375621 139703861815168 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f0ee040a160>: Note that this layer is not optimized for performance. Please use tf.keras.laye

[('input_encoder/forward_unified_lstm/kernel:0', TensorShape([300, 1200])),
 ('input_encoder/forward_unified_lstm/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('input_encoder/forward_unified_lstm/bias:0', TensorShape([1200])),
 ('input_encoder/backward_unified_lstm/kernel:0', TensorShape([300, 1200])),
 ('input_encoder/backward_unified_lstm/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('input_encoder/backward_unified_lstm/bias:0', TensorShape([1200])),
 ('feature_fc/kernel:0', TensorShape([2400, 300])),
 ('feature_fc/bias:0', TensorShape([300])),
 ('revise_encoder/forward_unified_lstm_1/kernel:0', TensorShape([300, 1200])),
 ('revise_encoder/forward_unified_lstm_1/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('revise_encoder/forward_unified_lstm_1/bias:0', TensorShape([1200])),
 ('revise_encoder/backward_unified_lstm_1/kernel:0', TensorShape([300, 1200])),
 ('revise_encoder/backward_unified_lstm_1/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('revise_encoder/bac

I0411 01:08:20.601633 139703861815168 interactiveshell.py:2882] Step 0 | Loss: 1.1463 | Spent: 41.0 secs | LR: 0.000400
I0411 01:08:31.707270 139703861815168 interactiveshell.py:2882] Step 50 | Loss: 1.2115 | Spent: 11.1 secs | LR: 0.000400
I0411 01:08:42.908444 139703861815168 interactiveshell.py:2882] Step 100 | Loss: 1.1444 | Spent: 11.2 secs | LR: 0.000400
I0411 01:08:54.111345 139703861815168 interactiveshell.py:2882] Step 150 | Loss: 1.0815 | Spent: 11.2 secs | LR: 0.000399
I0411 01:09:05.342963 139703861815168 interactiveshell.py:2882] Step 200 | Loss: 1.0911 | Spent: 11.2 secs | LR: 0.000399
I0411 01:09:16.686294 139703861815168 interactiveshell.py:2882] Step 250 | Loss: 1.0469 | Spent: 11.3 secs | LR: 0.000399
I0411 01:09:28.065764 139703861815168 interactiveshell.py:2882] Step 300 | Loss: 0.9715 | Spent: 11.4 secs | LR: 0.000399
I0411 01:09:39.090162 139703861815168 interactiveshell.py:2882] Step 350 | Loss: 1.0681 | Spent: 11.0 secs | LR: 0.000399
I0411 01:09:49.561542 13970

Reading ../data/test.txt


I0411 01:15:16.904093 139703861815168 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.700
I0411 01:15:16.905418 139703861815168 interactiveshell.py:2882] Best Accuracy: 0.700
I0411 01:15:24.458270 139703861815168 interactiveshell.py:2882] Step 1750 | Loss: 0.7055 | Spent: 43.9 secs | LR: 0.000393
I0411 01:15:35.807384 139703861815168 interactiveshell.py:2882] Step 1800 | Loss: 0.8493 | Spent: 11.3 secs | LR: 0.000393
I0411 01:15:47.180665 139703861815168 interactiveshell.py:2882] Step 1850 | Loss: 0.5320 | Spent: 11.4 secs | LR: 0.000393
I0411 01:15:58.559577 139703861815168 interactiveshell.py:2882] Step 1900 | Loss: 0.6986 | Spent: 11.4 secs | LR: 0.000392
I0411 01:16:10.022040 139703861815168 interactiveshell.py:2882] Step 1950 | Loss: 0.5331 | Spent: 11.5 secs | LR: 0.000392
I0411 01:16:21.162157 139703861815168 interactiveshell.py:2882] Step 2000 | Loss: 0.5089 | Spent: 11.1 secs | LR: 0.000392
I0411 01:16:32.438274 139703861815168 interactiveshell.py:2882] Step 2050 | L

...

Reading ../data/test.txt


I0411 09:02:59.041755 139703861815168 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.873
I0411 09:02:59.042971 139703861815168 interactiveshell.py:2882] Best Accuracy: 0.874
I0411 09:03:05.357044 139703861815168 interactiveshell.py:2882] Step 115200 | Loss: 0.2911 | Spent: 43.8 secs | LR: 0.000126
I0411 09:03:16.339867 139703861815168 interactiveshell.py:2882] Step 115250 | Loss: 0.3029 | Spent: 11.0 secs | LR: 0.000126
I0411 09:03:27.579362 139703861815168 interactiveshell.py:2882] Step 115300 | Loss: 0.3732 | Spent: 11.2 secs | LR: 0.000126
I0411 09:03:38.704532 139703861815168 interactiveshell.py:2882] Step 115350 | Loss: 0.5584 | Spent: 11.1 secs | LR: 0.000125
I0411 09:03:49.739296 139703861815168 interactiveshell.py:2882] Step 115400 | Loss: 0.3493 | Spent: 11.0 secs | LR: 0.000125
I0411 09:04:00.830431 139703861815168 interactiveshell.py:2882] Step 115450 | Loss: 0.2836 | Spent: 11.1 secs | LR: 0.000125
I0411 09:04:12.639321 139703861815168 interactiveshell.py:2882] S

Reading ../data/test.txt


I0411 09:09:54.297519 139703861815168 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.871
I0411 09:09:54.298831 139703861815168 interactiveshell.py:2882] Best Accuracy: 0.874
I0411 09:09:56.345757 139703861815168 interactiveshell.py:2882] Step 116900 | Loss: 0.4605 | Spent: 44.6 secs | LR: 0.000124
I0411 09:10:07.499701 139703861815168 interactiveshell.py:2882] Step 116950 | Loss: 0.2140 | Spent: 11.2 secs | LR: 0.000123
I0411 09:10:18.804139 139703861815168 interactiveshell.py:2882] Step 117000 | Loss: 0.3312 | Spent: 11.3 secs | LR: 0.000123
I0411 09:10:29.878822 139703861815168 interactiveshell.py:2882] Step 117050 | Loss: 0.3115 | Spent: 11.1 secs | LR: 0.000123


KeyboardInterrupt: ignored